## Import required packages

In [1]:
import numpy as np 
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import json

In [2]:
!ls

README.md  bot_env  data  requirements.txt  resources  scripts


## Authenticate with Spotify API

In [3]:
# Credentials are specified in the "activate" script from the venv and can be obtained in the Spotify Developer Platform
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
### Show playlists to check if connection works

In [5]:
playlists = sp.user_playlists('arthevard')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

   1 spotify:playlist:6pm6NYE8x95f3qu9nnkY3s Gestört
   2 spotify:playlist:3w4J1cBM7G8A3FJpmkQCks Jump Rope Fun
   3 spotify:playlist:22F1DzV6R5WTtaU5ZBx1Qh Alman Rap
   4 spotify:playlist:7sRH3hcKY9vi5Om9hKWED6 Dancehall Excellence
   5 spotify:playlist:7bKCXksqUpwO2u77DNDB6E Schweche Excellence
   6 spotify:playlist:2NL4skzzhfDIWGexthIoti Cloudrap und sowas
   7 spotify:playlist:45jLD4C7zDnVwmrs9Q2Yst Chill Excellence
   8 spotify:playlist:7JLpMUShki1OtQBbq0SJSD Spulung
   9 spotify:playlist:6UcKBh9vtZOGIMEx1FGXGj Hip Hop Excellence
  10 spotify:playlist:4IksObf8ZWrLkyWhcOp1Vp Classic Excellence
  11 spotify:playlist:62jUJdEWgsCmrh2EpCABRG Rock Excellence
  12 spotify:playlist:5QKkIeuRrBQ2L501fAi5vy Melancholy Excellence
  13 spotify:playlist:7i2EfrY01tIbPZ81sQXKKE Nostalgia Excellence
  14 spotify:playlist:5mJ07XW8fizydNB9mqRzKk Zu Arg für Bib LEL²
  15 spotify:playlist:48sxmNjqMplmnbSjnky8Tz Memes


In [6]:

urn = 'spotify:artist:3NjbpG6MmFGVLXwbcPXH90'
artist = sp.artist(urn)
print(json.dumps(artist, indent=2))



{
  "external_urls": {
    "spotify": "https://open.spotify.com/artist/3NjbpG6MmFGVLXwbcPXH90"
  },
  "followers": {
    "href": null,
    "total": 62665
  },
  "genres": [
    "frauenrap",
    "german cloud rap",
    "german drill",
    "german hip hop",
    "indie deutschrap"
  ],
  "href": "https://api.spotify.com/v1/artists/3NjbpG6MmFGVLXwbcPXH90",
  "id": "3NjbpG6MmFGVLXwbcPXH90",
  "images": [
    {
      "height": 640,
      "url": "https://i.scdn.co/image/4115e7cde63d48bbda9cfc1e9aeb83efc4b720c8",
      "width": 640
    },
    {
      "height": 320,
      "url": "https://i.scdn.co/image/1a600d30661d2d2cf6d087964a15033aa65599a3",
      "width": 320
    },
    {
      "height": 160,
      "url": "https://i.scdn.co/image/3d5a6e4c11865aff5f08046820d24696e411ba04",
      "width": 160
    }
  ],
  "name": "Haiyti",
  "popularity": 55,
  "type": "artist",
  "uri": "spotify:artist:3NjbpG6MmFGVLXwbcPXH90"
}


# Import users and artists URIs from CSV files

In [75]:
user_uri_list = pd.read_csv("data/users.csv", sep=";")["user_uri"]
artist_uri_list = pd.read_csv("data/artists.csv", sep=";")["artist_uri"]

In [76]:
user = sp.user(user_uri_list[0])
user

{'display_name': 'Carsten We',
 'external_urls': {'spotify': 'https://open.spotify.com/user/arthevard'},
 'followers': {'href': None, 'total': 21},
 'href': 'https://api.spotify.com/v1/users/arthevard',
 'id': 'arthevard',
 'images': [{'height': None,
   'url': 'https://scontent.flux1-1.fna.fbcdn.net/v/t1.6435-1/p320x320/36594189_10214425521425989_1333356851693092864_n.jpg?_nc_cat=110&ccb=1-3&_nc_sid=0c64ff&_nc_ohc=Wjoor2MFCncAX9_svCX&_nc_ht=scontent.flux1-1.fna&tp=6&oh=bcf35634f57f710a24f1ebe907a8e89f&oe=609F611D',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:arthevard'}

## Write basic functions to extract data from artists

In [43]:
def get_songs_from_albums(album_list):

    for album in album_list:
        sui_sui_json = sp.album_tracks(album_list[album], limit=50, offset=0)
        tracklist = sui_sui_json["items"]

        for track in range(len(tracklist)):
            track_name = tracklist[track]["name"]
            print(track_name)


In [62]:
def get_albums_from_artists(artist_uri_list):
    
    # Create header for output df
    albums_list = [['name', 'album_uri', 'artist_uri']]

    # Loop through list of artist uris
    for artist_uri in artist_uri_list:
        # Get album from artist
        albums = sp.artist_albums(artist_uri)
        
        # Append each album to list
        for album in albums["items"]:
            album_name = album["name"]
            album_uri = album["uri"]
            albums_list.append([album_name, album_uri, artist_uri])

    # Create df from list of albums for all artist
    albums_df = pd.DataFrame(data=albums_list[1:], columns=albums_list[0])

    return albums_df


In [74]:
get_albums_from_artists(artist_uri_list)

,name,album_uri,artist_uri
0,MIESES LEBEN,spotify:album:18iDtijC0YUY43T2teLEiK,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
1,influencer,spotify:album:7MguPrKUh0eyZY0VqmMO0G,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
2,SUI SUI,spotify:album:2M51xiotVFca2hT7OppmgK,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
3,Nightliner Reloaded,spotify:album:5KYZEBBBwDPZ9UY9Tpr3E0,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
4,Perroquet,spotify:album:5h5iAcFwOtsKZJZRMCZ8HV,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
5,ATM,spotify:album:6wsNMSecBx8FXB4FlSga5p,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
6,Montenegro Zero,spotify:album:3JvHhoB5qndBlQFlTHfPIa,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
7,Follow mich nicht,spotify:album:0JFOxeUNwyOsfXG1IBLGQO,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
8,City Tarif,spotify:album:6YEWMwhymNbTEatqBJXD00,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
9,FREITAG (The Belgian Stallion Remix),spotify:album:5tfx4YsMa9gMalKg4gkk3I,spotify:artist:3NjbpG6MmFGVLXwbcPXH90
